In [13]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.feature_selection import mutual_info_regression, SelectPercentile, SelectKBest, RFE

### Pre-processing

In [14]:
# Reading the Training Data
df = pd.read_csv("C:/Users/kvsvy/Downloads/traffic volume data/train_set_dirty.csv")

In [15]:
# 1. Displaying the first 10 records
# 1. Displaying the first 10 records
# 2. Check dataset info - field types, non-null values, dtypes
# df.shape
# df.columns
# df.dtypes
# df.describe()
df.info()
# df.nunique()
# df['weather_description'].unique()
# df.tail(5)
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38563 entries, 0 to 38562
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   traffic_volume       34748 non-null  float64
 1   holiday              42 non-null     object 
 2   temp                 34754 non-null  float64
 3   rain_1h              34860 non-null  float64
 4   snow_1h              34733 non-null  float64
 5   clouds_all           34659 non-null  float64
 6   weather_main         34654 non-null  object 
 7   weather_description  34751 non-null  object 
 8   date_time            34683 non-null  object 
dtypes: float64(5), object(4)
memory usage: 2.6+ MB


traffic_volume          3815
holiday                38521
temp                    3809
rain_1h                 3703
snow_1h                 3830
clouds_all              3904
weather_main            3909
weather_description     3812
date_time               3880
dtype: int64

In [17]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%d-%m-%Y %H:%M')
df['date_month_no']= df['date_time'].dt.month
df['date_day']= df['date_time'].dt.day
df['date_hour']= df['date_time'].dt.hour
df['weekday'] = df['date_time'].dt.weekday
df['is_weekend'] = df['date_time'].dt.dayofweek // 5
df['hour_sin'] = np.sin(2 * np.pi * df['date_hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['date_hour'] / 24)
df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
df['date_day_sin'] = np.sin(2 * np.pi * df['date_day'] / 31)
df['date_day_cos'] = np.cos(2 * np.pi * df['date_day'] / 31)
df['day_of_year'] = df['date_time'].dt.dayofyear
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)

In [5]:
# 2. Check dataset info - field types, non-null values, dtypes

In [18]:
# 3a. Pre-processing (Cleaning): Address missing (NULL) values - drop or imputation
values = {
    'temp': df['temp'].mean(),
    'rain_1h': df['rain_1h'].interpolate(method='linear'),
    'snow_1h': df['snow_1h'].interpolate(method='linear'),
    'clouds_all': df['clouds_all'].mean()


}
df.fillna(value=values, inplace=True)
df.dropna(subset= ['traffic_volume','weather_main','weather_description','date_time'], inplace=True)

In [19]:
# 3c. Pre-processing (Cleaning): Drop irrelevant columns (that you don't want to give as input to the model)
df = df.drop(columns=["holiday","date_time"])

In [20]:
# 3d. Pre-processing (Encoding): Convert categorical values to numeric
# Use df['col_name'].value_counts() to find out all the categories available per column

# df['col_name'].value_counts()

df = pd.get_dummies(df, columns=['weather_main','weather_description']).astype(float) 

In [21]:
# in order to match hotcoding with test data i am manually deleting those unavailable in test data
# df = df.drop(columns=["holiday","date_time"])
df= df.drop(columns=["weather_description_light rain and snow","weather_description_shower snow","weather_description_thunderstorm with drizzle","weather_description_sleet"])

In [22]:
# 3b. Pre-processing (Cleaning): Remove Duplicate Rows
df.drop_duplicates(inplace=True)

In [23]:
# 4a. Data understanding - Find out stats regarding your data (df.describe(), df.mean(), df.median())
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 25327 entries, 3 to 38562
Data columns (total 64 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   traffic_volume                                           25327 non-null  float64
 1   temp                                                     25327 non-null  float64
 2   rain_1h                                                  25327 non-null  float64
 3   snow_1h                                                  25327 non-null  float64
 4   clouds_all                                               25327 non-null  float64
 5   date_month_no                                            25327 non-null  float64
 6   date_day                                                 25327 non-null  float64
 7   date_hour                                                25327 non-null  float64
 8   weekday                        

traffic_volume                                         0
temp                                                   0
rain_1h                                                0
snow_1h                                                0
clouds_all                                             0
                                                      ..
weather_description_thunderstorm with heavy rain       0
weather_description_thunderstorm with light drizzle    0
weather_description_thunderstorm with light rain       0
weather_description_thunderstorm with rain             0
weather_description_very heavy rain                    0
Length: 64, dtype: int64

In [ ]:
# 4b. Data understanding - Make use of plots to build more understanding of the data
# Hint: Can use df.plot()
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate correlation matrix
corr = df[features + ['traffic_volume']].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
sns.pairplot(df[features + ['traffic_volume']])
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Assuming you have already trained a RandomForestRegressor model named 'rf'
feature_importances = selector.feature_importances_
sorted_idx = np.argsort(feature_importances)

plt.figure(figsize=(16, 7))
plt.barh(range(len(selected_features_names)), feature_importances[selected_features_names], align='center')
plt.yticks(range(len(selected_features_names)), np.array(features)[selected_features_names])
plt.title('Feature Importances')
plt.show()

In [ ]:
# 4b. (Optional) Create More plots to understand the relationship b/w different variables

# Example: Top 20 actors of movies based on the imdb rating of the movies

# plt.figure(figsize=(10, 8))

# # Create a new dataframe with top 20 values
# new_df = df.sort_values(by ='imdb_score' , ascending=False)
# new_df = new_df.head(20)

# # plotting
# ax=sns.pointplot(x=new_df['actor_1_name'], y=new_df['imdb_score'], hue=new_df['movie_title'])
# ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
# plt.tight_layout()
# plt.show()

In [ ]:
# 4c. Find out which input features are the most important
# Hint: Start out with df.corr(). Can visualise with seaborn library

In [ ]:
# Any other relevant pre-processing (upto your exploration)

### Model Development

In [24]:
# 1. Divide dataframe into input and output
# X = df.drop(columns=['output_class']) -> Drop the column to be predicted
# y = df['output_class'] -> Choose Output column to be predicted
x = df[[ 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main_Clear',
       'weather_main_Clouds', 'weather_main_Drizzle', 'weather_main_Fog',
       'weather_main_Haze', 'weather_main_Mist', 'weather_main_Rain',
       'weather_main_Smoke', 'weather_main_Snow', 'weather_main_Squall',
       'weather_main_Thunderstorm', 'weather_description_SQUALLS',
       'weather_description_Sky is Clear', 'weather_description_broken clouds',
       'weather_description_drizzle', 'weather_description_few clouds',
       'weather_description_fog', 'weather_description_freezing rain',
       'weather_description_haze',
       'weather_description_heavy intensity drizzle',
       'weather_description_heavy intensity rain',
       'weather_description_heavy snow',
       'weather_description_light intensity drizzle',
       'weather_description_light intensity shower rain',
       'weather_description_light rain',
       'weather_description_light shower snow',
       'weather_description_light snow', 'weather_description_mist',
       'weather_description_moderate rain',
       'weather_description_overcast clouds',
       'weather_description_proximity shower rain',
       'weather_description_proximity thunderstorm',
       'weather_description_proximity thunderstorm with drizzle',
       'weather_description_proximity thunderstorm with rain',
       'weather_description_scattered clouds',
       'weather_description_shower drizzle',
       'weather_description_sky is clear', 'weather_description_smoke',
       'weather_description_snow', 'weather_description_thunderstorm',
       'weather_description_thunderstorm with heavy rain',
       'weather_description_thunderstorm with light drizzle',
       'weather_description_thunderstorm with light rain',
       'weather_description_thunderstorm with rain',
       'weather_description_very heavy rain','date_month_no','date_hour',
       'date_day','weekday','is_weekend','hour_sin','hour_cos','weekday_sin','weekday_cos',
       'day_of_year_sin','day_of_year_cos','day_of_year']]
y = df['traffic_volume']
feature_names = x.columns



In [51]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Feature Selection with Random Forest
selector = RandomForestRegressor(n_estimators=100, random_state=42)
selector.fit(x, y)

# Get feature importances
feature_importances = selector.feature_importances_

# Get indices of the top 6 features
selected_features_indices = (-feature_importances).argsort()[:10]

# Create a new DataFrame with only the selected features
x_new = x.iloc[:, selected_features_indices]

# Get the names of the selected features
selected_features_names = x.columns[selected_features_indices]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size=0.2)

# Train the XGBoost Regressor model
model = XGBRegressor()
model.fit(x_train, y_train)

# Make predictions
y_pred = model.predict(x_test)

# Evaluate the model
print('R^2 Score:', r2_score(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))


R^2 Score: 0.9730103732869331
Mean Squared Error: 104376.23967654924


In [52]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Assuming x_train, y_train are your training data and labels

# Hyperparameter tuning
parameters = {
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1]
}

xgb = XGBRegressor()
clf = GridSearchCV(xgb, parameters, scoring='neg_mean_squared_error')
clf.fit(x_train, y_train)

# Use the best parameters for the model
model = clf.best_estimator_

# Optionally, re-evaluate feature importance and select top features
# model.fit(x_train[selected_features], y_train)
# predictions = model.predict(x_test[selected_features])

# Evaluate the model

In [53]:
# Assuming 'model' is your updated model after hyperparameter tuning or feature selection
model.fit(x_train, y_train)  # Retrain the model with the training data

y_pred = model.predict(x_test)  # Make predictions on the test data

# Calculate the Mean Squared Error to evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 95927.66244425617


In [59]:
selected_features_names



Index(['hour_cos', 'date_hour', 'hour_sin', 'is_weekend', 'weekday_sin',
       'weekday', 'day_of_year_cos', 'day_of_year_sin', 'temp', 'day_of_year'],
      dtype='object')

In [ ]:
# 3. Data Normalisation: Bring into the range 0 to 1, or -1 to 1

In [ ]:
# 4. Choose Model(s), fit
### Experiment with different models.
### https://scikit-learn.org/stable/supervised_learning.html
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'df' is your DataFrame and it contains both the features and the target variable
features = ['hour_cos', 'date_hour', 'hour_sin', 'is_weekend', 'weekday_sin',
            'weekday', 'day_of_year_cos', 'day_of_year_sin', 'temp', 'day_of_year']
target = 'traffic_volume'

# Set the size of the plots
plt.figure(figsize=(20, 15))

for i, feature in enumerate(features, 1):
    plt.subplot(5, 2, i)  # Adjust the grid size according to the number of features
    sns.scatterplot(x=df[feature], y=df[target])
    plt.title(f'{feature} vs. {target}')
    plt.xlabel(feature)
    plt.ylabel(target)

plt.tight_layout()
plt.show()

In [ ]:
pip install matplotlib seaborn

In [ ]:
plt.figure(figsize=(20, 15))
for i, feature in enumerate(features, 1):
    plt.subplot(5, 2, i)
    # Sorting values for better line plot visualization
    sorted_df = df.sort_values(by=feature)
    plt.plot(sorted_df[feature], sorted_df[target], marker='o', linestyle='-', markersize=4)
    plt.title(f'{feature} vs. {target}')
    plt.xlabel(feature)
    plt.ylabel(target)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
for i, feature in enumerate(features, 1):
    plt.subplot(5, 2, i)
    sns.boxplot(x=df[feature], y=df[target])
    plt.title(f'{feature} vs. {target}')
    plt.xlabel(feature)
    plt.ylabel(target)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
for i, feature in enumerate(features, 1):
    plt.subplot(5, 2, i)
    sns.violinplot(x=df[feature], y=df[target])
    plt.title(f'{feature} vs. {target}')
    plt.xlabel(feature)
    plt.ylabel(target)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
for i, feature in enumerate(features, 1):
    plt.subplot(5, 2, i)
    plt.hist(df[feature], bins=20, alpha=0.7)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# 5. Evaluate with relevant metric for your problem. Eg: accuracy_score(), r2_score()

In [ ]:
# 6. After model choice is made, fine-tune with GridSearchCV, or RandomizedSearchCV()

### Testing and Creating Output CSV

In [55]:
# Creating output file for submission - Template Code

test_pd = pd.read_csv("C:/Users/kvsvy/Downloads/traffic volume data/test_set_nogt.csv")

# saving in a new variable to modify
test = test_pd.copy(deep=True)

test['date_time'] = pd.to_datetime(test['date_time'], format='%d-%m-%Y %H:%M')
test['date_month_no']= test['date_time'].dt.month
test['date_day']= test['date_time'].dt.day
test['date_hour']= test['date_time'].dt.hour
test['weekday'] = test['date_time'].dt.weekday
test['is_weekend'] = test['date_time'].dt.dayofweek // 5
test['hour_sin'] = np.sin(2 * np.pi * test['date_hour'] / 24)
test['hour_cos'] = np.cos(2 * np.pi * test['date_hour'] / 24)
test['weekday_sin'] = np.sin(2 * np.pi * test['weekday'] / 7)
test['weekday_cos'] = np.cos(2 * np.pi * test['weekday'] / 7)
test['date_day_sin'] = np.sin(2 * np.pi * test['date_day'] / 31)
test['date_day_cos'] = np.cos(2 * np.pi * test['date_day'] / 31)
test['day_of_year'] = test['date_time'].dt.dayofyear
test['day_of_year_sin'] = np.sin(2 * np.pi * test['day_of_year'] / 365.25)
test['day_of_year_cos'] = np.cos(2 * np.pi * test['day_of_year'] / 365.25)
test.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9641 entries, 0 to 9640
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   holiday              16 non-null     object        
 1   temp                 9641 non-null   float64       
 2   rain_1h              9641 non-null   float64       
 3   snow_1h              9641 non-null   float64       
 4   clouds_all           9641 non-null   int64         
 5   weather_main         9641 non-null   object        
 6   weather_description  9641 non-null   object        
 7   date_time            9641 non-null   datetime64[ns]
 8   date_month_no        9641 non-null   int32         
 9   date_day             9641 non-null   int32         
 10  date_hour            9641 non-null   int32         
 11  weekday              9641 non-null   int32         
 12  is_weekend           9641 non-null   int32         
 13  hour_sin             9641 non-nul

In [56]:
test = test.drop(columns=["holiday","date_time"])
test = pd.get_dummies(test, columns=['weather_main','weather_description']).astype(float) 
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9641 entries, 0 to 9640
Data columns (total 63 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   temp                                                     9641 non-null   float64
 1   rain_1h                                                  9641 non-null   float64
 2   snow_1h                                                  9641 non-null   float64
 3   clouds_all                                               9641 non-null   float64
 4   date_month_no                                            9641 non-null   float64
 5   date_day                                                 9641 non-null   float64
 6   date_hour                                                9641 non-null   float64
 7   weekday                                                  9641 non-null   float64
 8   is_weekend                  

In [57]:

# # Prepare data to be given as an input to your trained model
# # 1. Repeat the pre-processing done above. Eg: Conversion to categorical, filling in mean values
# test['bmi'].fillna(test['bmi'].mean(), inplace=True)
# test = pd.get_dummies(test)

# 2. Use the same features obtained in feature selection
# chosen_features = feat_selector.get_feature_names_out() # from above -> getting names of chosen features
test = test[selected_features_names]

# 3. Normalise/Scale the features as done above


# 4. Predict and obtain results from the model
y_pred = model.predict(test)

# 5. Save results to CSV
submission = pd.DataFrame({'ID': test.index, 'traffic_volume' : y_pred})
# submission.to_csv('submissionpost.csv', index=False)
submission.to_csv('C:/Users/kvsvy/Downloads/submissionpostfinal.csv', index=False)